In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from fastcore.parallel import parallel
from fastai.tabular.all import df_shrink

In [14]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)

In [ ]:
 
FOLDER_PATH = '/Users/srinara2/Downloads/Dissertation_research/cic-ids2017-ml/'

PARQUET_OUTPUT_PATH = '/Users/srinara2/Downloads/Dissertation_research/cic-ids2017-ml/parquet_output'
EXCLUDE_FILES = ['Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv']

KNOWN_HEADER_VALUES = [
    'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts',
    'Flow Duration', 'Fwd Pkt Len Max', 'Bwd Pkt Len Mean', 'Protocol'
]

COL_NAME_CONSISTENCY = {
    'Flow ID': 'Flow ID', 'Source IP': 'Source IP', 'Src IP':  'Source IP',
    'Source Port': 'Source Port', 'Src Port': 'Source Port',
    'Destination IP': 'Destination IP', 'Dst IP': 'Destination IP',
    'Destination Port': 'Destination Port', 'Dst Port': 'Destination Port',
    'Protocol': 'Protocol', 'Timestamp': 'Timestamp',
    'Flow Duration': 'Flow Duration', 'Total Fwd Packets': 'Total Fwd Packets',
    'Tot Fwd Pkts': 'Total Fwd Packets', 'Total Backward Packets': 'Total Backward Packets',
    'Tot Bwd Pkts': 'Total Backward Packets',
    'Total Length of Fwd Packets': 'Fwd Packets Length Total', 'TotLen Fwd Pkts': 'Fwd Packets Length Total',
    'Total Length of Bwd Packets': 'Bwd Packets Length Total', 'TotLen Bwd Pkts': 'Bwd Packets Length Total',
    'Fwd Packet Length Max': 'Fwd Packet Length Max', 'Fwd Pkt Len Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min', 'Fwd Pkt Len Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean', 'Fwd Pkt Len Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std', 'Fwd Pkt Len Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max', 'Bwd Pkt Len Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min', 'Bwd Pkt Len Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean', 'Bwd Pkt Len Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std', 'Bwd Pkt Len Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s', 'Flow Byts/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s', 'Flow Pkts/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean', 'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max', 'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total', 'Fwd IAT Tot': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean', 'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max', 'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total', 'Bwd IAT Tot': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean', 'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max', 'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd PSH Flags': 'Fwd PSH Flags', 'Bwd PSH Flags': 'Bwd PSH Flags',
    'Fwd URG Flags': 'Fwd URG Flags', 'Bwd URG Flags': 'Bwd URG Flags',
    'Fwd Header Length': 'Fwd Header Length', 'Fwd Header Len': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length', 'Bwd Header Len': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s', 'Fwd Pkts/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s', 'Bwd Pkts/s': 'Bwd Packets/s',
    'Min Packet Length': 'Packet Length Min', 'Pkt Len Min': 'Packet Length Min',
    'Max Packet Length': 'Packet Length Max', 'Pkt Len Max': 'Packet Length Max',
    'Packet Length Mean': 'Packet Length Mean', 'Pkt Len Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std', 'Pkt Len Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance', 'Pkt Len Var': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count', 'FIN Flag Cnt': 'FIN Flag Count',
    'SYN Flag Count': 'SYN Flag Count', 'SYN Flag Cnt': 'SYN Flag Count',
    'RST Flag Count': 'RST Flag Count', 'RST Flag Cnt': 'RST Flag Count',
    'PSH Flag Count': 'PSH Flag Count', 'PSH Flag Cnt': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count', 'ACK Flag Cnt': 'ACK Flag Count',
    'URG Flag Count': 'URG Flag Count', 'URG Flag Cnt': 'URG Flag Count',
    'CWE Flag Count': 'CWE Flag Count', 'CWE Flag Cnt': 'CWE Flag Count',
    'ECE Flag Count': 'ECE Flag Count', 'ECE Flag Cnt': 'ECE Flag Count',
    'Down/Up Ratio': 'Down/Up Ratio', 'Average Packet Size': 'Avg Packet Size',
    'Pkt Size Avg': 'Avg Packet Size', 'Avg Fwd Segment Size': 'Avg Fwd Segment Size',
    'Fwd Seg Size Avg': 'Avg Fwd Segment Size', 'Avg Bwd Segment Size': 'Avg Bwd Segment Size',
    'Bwd Seg Size Avg': 'Avg Bwd Segment Size',
    'Fwd Avg Bytes/Bulk': 'Fwd Avg Bytes/Bulk', 'Fwd Byts/b Avg': 'Fwd Avg Bytes/Bulk',
    'Fwd Avg Packets/Bulk': 'Fwd Avg Packets/Bulk', 'Fwd Pkts/b Avg': 'Fwd Avg Packets/Bulk',
    'Fwd Avg Bulk Rate': 'Fwd Avg Bulk Rate', 'Fwd Blk Rate Avg': 'Fwd Avg Bulk Rate',
    'Bwd Avg Bytes/Bulk': 'Bwd Avg Bytes/Bulk', 'Bwd Byts/b Avg': 'Bwd Avg Bytes/Bulk',
    'Bwd Avg Packets/Bulk': 'Bwd Avg Packets/Bulk', 'Bwd Pkts/b Avg': 'Bwd Avg Packets/Bulk',
    'Bwd Avg Bulk Rate': 'Bwd Avg Bulk Rate', 'Bwd Blk Rate Avg': 'Bwd Avg Bulk Rate',
    'Subflow Fwd Packets': 'Subflow Fwd Packets', 'Subflow Fwd Pkts': 'Subflow Fwd Packets',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes', 'Subflow Fwd Byts': 'Subflow Fwd Bytes',
    'Subflow Bwd Packets': 'Subflow Bwd Packets', 'Subflow Bwd Pkts': 'Subflow Bwd Packets',
    'Subflow Bwd Bytes': 'Subflow Bwd Bytes', 'Subflow Bwd Byts': 'Subflow Bwd Bytes',
    'Init_Win_bytes_forward': 'Init Fwd Win Bytes', 'Init Fwd Win Byts': 'Init Fwd Win Bytes',
    'Init_Win_bytes_backward': 'Init Bwd Win Bytes', 'Init Bwd Win Byts': 'Init Bwd Win Bytes',
    'act_data_pkt_fwd': 'Fwd Act Data Packets', 'Fwd Act Data Pkts': 'Fwd Act Data Packets',
    'min_seg_size_forward': 'Fwd Seg Size Min', 'Fwd Seg Size Min': 'Fwd Seg Size Min',
    'Active Mean': 'Active Mean', 'Active Std': 'Active Std', 'Active Max': 'Active Max',
    'Active Min': 'Active Min', 'Idle Mean': 'Idle Mean', 'Idle Std': 'Idle Std',
    'Idle Max': 'Idle Max', 'Idle Min': 'Idle Min', 'Label': 'Label'
}

DROP_COLUMNS = [
    "Flow ID", 'Fwd Header Length.1',
    "Source IP", "Src IP", "Source Port", "Src Port",
    "Destination IP", "Dst IP", 
    "Timestamp", # Dropping here, but consider extracting features first if needed
]


In [16]:
def load_dataframes(folder_path: str, exclude_files: list = None) -> dict:
    """Loads CSV files from a specified folder into a dictionary of DataFrames."""
    if exclude_files is None:
        exclude_files = []

    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and f not in exclude_files]
    dfs = {}
    for file in csv_files:
        df_name = os.path.splitext(file)[0]
        file_path = os.path.join(folder_path, file)
        try:
            df = pd.read_csv(file_path, low_memory=False)
            dfs[df_name] = df
            print(f"Loaded '{df_name}' with shape {df.shape}")
        except Exception as e:
            print(f"âŒ Error loading {file}: {e}")
    return dfs

In [17]:
def remove_fake_header_rows(dfs: dict, known_values: list) -> dict:
    """Removes rows suspected to be fake headers based on known column names."""
    lower_knowns = [v.strip().lower() for v in known_values]
    cleaned_dfs = {}
    for name, df in dfs.items():
        print(f"\n🧹 Cleaning fake headers in: {name}")
        before = len(df)
        mask = df.apply(lambda row: any(str(cell).strip().lower() in lower_knowns for cell in row), axis=1)
        df_cleaned = df[~mask].reset_index(drop=True) # Avoid inplace here
        cleaned_dfs[name] = df_cleaned
        print(f"✅ Removed {before - len(df_cleaned)} suspected header rows")
    return cleaned_dfs

In [18]:
def convert_object_columns_to_numeric(dfs: dict) -> dict:
    """Converts object columns (except Timestamp and Label) to numeric, coercing errors."""
    processed_dfs = {}
    for name, df in dfs.items():
        # Make a copy to avoid SettingWithCopyWarning if original df is a slice
        df_copy = df.copy()
        df_copy.columns = df_copy.columns.str.strip()  # ensure clean column names

         
        print(f"\nConverting columns in {name} to numeric:")
        for col in df_copy.columns:
            if col not in ['Timestamp', 'Label'] and df_copy[col].dtype == 'object':
                print(f" -  {col}")
                df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')
                
        processed_dfs[name] = df_copy
    return processed_dfs

In [19]:
def apply_column_consistency_and_drop(dfs: dict, col_map: dict, drop_cols: list) -> dict:
    """Applies consistent column names and drops specified columns."""
    processed_dfs = {}
    for name, df in dfs.items():
        df_copy = df.copy() # Work on a copy

        df_copy.columns = df_copy.columns.str.strip()
        df_copy = df_copy.drop(columns=drop_cols, errors='ignore') # Assign back
        df_copy = df_copy.rename(columns=col_map) # Assign back

        # Fix label values (ensure this matches the target name in col_map)
        if 'Label' in df_copy.columns:
            df_copy['Label'] = df_copy['Label'].replace({'BENIGN': 'Benign'}) # Assign back

        processed_dfs[name] = df_copy
    return processed_dfs

In [20]:
def handle_inf_nan_duplicates(dfs: dict) -> dict:
    """Replaces inf/-inf with NaN, drops NaNs, and removes duplicate rows."""
    processed_dfs = {}
    for name, df in dfs.items():
        print(f"Dataframe: '{name}'\n")

        # Replace inf/-inf with NaN
        df_copy = df.replace([np.inf, -np.inf], np.nan) # Assign back

        print(f"Before dropping NaNs: {df_copy.isna().any(axis=1).sum()} rows with at least one NaN")
        df_copy = df_copy.dropna() # Assign back
        print(f"Remaining rows after dropping NaNs: {len(df_copy)}")

        print(f"{df_copy.duplicated().sum()} fully duplicate rows to remove")
        df_copy = df_copy.drop_duplicates().reset_index(drop=True) # Assign back
        print(f"Rows after dropping duplicates: {len(df_copy)}")

        processed_dfs[name] = df_copy
        print("\n" + "-"*60 + "\n")
    return processed_dfs


In [21]:

def drop_constant_columns(df, df_name=""): # Added df_name parameter for better logging
    """
    Drops columns that have only one unique value,
    but explicitly keeps the 'Label' column if it exists.
    Prints which columns are dropped.
    """
    initial_cols = set(df.columns)
    
    # Identify non-constant columns (excluding 'Label' from this check)
    non_constant_cols = []
    constant_cols_to_drop = []

    for col in df.columns:
        if col == 'Label':
            non_constant_cols.append(col) # Always keep Label
        elif df[col].nunique(dropna=False) <= 1: # <=1 to catch truly constant and all-NaN columns
            constant_cols_to_drop.append(col)
        else:
            non_constant_cols.append(col)
            
    # Create the new DataFrame with only the desired columns
    df_cleaned = df[non_constant_cols]

    dropped_cols = initial_cols - set(df_cleaned.columns)

    if dropped_cols:
        print(f"  For DataFrame '{df_name}', dropped {len(dropped_cols)} constant columns: {list(dropped_cols)}")
    else:
        print(f"  For DataFrame '{df_name}', no constant columns were dropped.")

    return df_cleaned



In [22]:
def preprocess_cic_ids_data(
    data_folder: str,
    output_folder: str,
    known_header_values: list,
    col_name_consistency_map: dict,
    columns_to_drop: list,
    exclude_files_from_load: list = None
):
    """
    Orchestrates the entire CIC-IDS data preprocessing pipeline.
    """
    print("Starting data loading...")
    dfs = load_dataframes(data_folder, exclude_files_from_load)

    print("\n--- Initial Data Types ---")
    for name, df in dfs.items():
        print(f"\n{name} DataFrame dtypes:\n")
        print(df.dtypes)

    print("\nStarting fake header removal...")
    dfs = remove_fake_header_rows(dfs, known_header_values)

    print("\nStarting object column conversion to numeric...")
    dfs = convert_object_columns_to_numeric(dfs)

    print("\n--- Data Types after Numeric Conversion ---")
    for name, df in dfs.items():
        print(f"\n{name} DataFrame dtypes:\n")
        print(df.dtypes)

    print("\nChecking for remaining non-numeric entries...")
    for name, df in dfs.items():
        print(f"\n📂 Dataset: {name}")
        issues = find_non_numeric_entries(df) # Assuming find_non_numeric_entries is defined elsewhere
        if not issues:
            print("✅ All object columns are clean or truly non-numeric.")
        else:
            for col, samples in issues.items():
                print(f"🔧 {col}: {len(samples)} bad value(s) ➤ {samples}")


    print("\nApplying column consistency and dropping irrelevant columns...")
    dfs = apply_column_consistency_and_drop(dfs, col_name_consistency_map, columns_to_drop)

    print("\nStarting Inf/NaN handling and duplicate removal...")
    dfs = handle_inf_nan_duplicates(dfs)

    print("\n--- Data Types after Cleaning ---")
    for name, df in dfs.items():
        print(f"\n{name} DataFrame dtypes:\n")
        print(df.dtypes)

    print("\nShrinking DataFrames for memory optimization...")
    shrunken_values = parallel(f=df_shrink, items=dfs.values(), progress=True)
    dfs = dict(zip(dfs.keys(), shrunken_values))
    print("DataFrames shrunk. Memory usage:")
    for name, df in dfs.items():
        print(f"{name}: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    print("\nDropping constant columns...")
    dfs = {name: drop_constant_columns(df) for name, df in dfs.items()}
    print("Shapes after dropping constant columns:")
    for name, shape in zip(dfs.keys(), [df.shape for df in dfs.values()]):
        print(f"{name}: {shape}")

    print("\n--- Final Data Types ---")
    for name, df in dfs.items():
        print(f"\n{name} DataFrame dtypes:\n")
        print(df.dtypes)

    print("\nSaving processed DataFrames to Parquet...")
    os.makedirs(output_folder, exist_ok=True)
    for name, df in dfs.items():
        parquet_path = os.path.join(output_folder, f"{name}.parquet")
        df.to_parquet(parquet_path, index=False)
        print(f"Saved '{name}' to '{parquet_path}'")
    print("All processed DataFrames saved successfully.")

In [23]:
print("✅  All object columns are clean or truly non-numeric.")


✅  All object columns are clean or truly non-numeric.


In [24]:
# --- Main execution block ---
if __name__ == "__main__":
    # Mount Google Drive if in Colab
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        # Optional: verify drive contents
        # !ls "{FOLDER_PATH}"
    except ImportError:
        print("Not running in Google Colab. Skipping drive mount.")

    # You would also need to define find_non_numeric_entries function here or import it
    def find_non_numeric_entries(df, max_samples=5):
        report = {}
        for col in df.select_dtypes(include='object').columns:
            converted = pd.to_numeric(df[col], errors='coerce')
            mask = converted.isna() & df[col].notna()
            bad_values = df.loc[mask, col].unique().tolist()
            if bad_values:
                report[col] = bad_values[:max_samples]
        return report

    preprocess_cic_ids_data(
        data_folder=FOLDER_PATH,
        output_folder=PARQUET_OUTPUT_PATH,
        known_header_values=KNOWN_HEADER_VALUES,
        col_name_consistency_map=COL_NAME_CONSISTENCY,
        columns_to_drop=DROP_COLUMNS,
        exclude_files_from_load=EXCLUDE_FILES
    )


Not running in Google Colab. Skipping drive mount.
Starting data loading...
Loaded 'DDoS-Friday-WorkingHours-Afternoon.pcap_ISCX' with shape (225745, 84)
Loaded 'WebAttacks-Thursday-WorkingHours-Morning.pcap_ISCX' with shape (170366, 84)
Loaded 'Benign-Monday-WorkingHours.pcap_ISCX' with shape (529918, 84)
Loaded 'Bruteforce-Tuesday-WorkingHours.pcap_ISCX' with shape (445909, 84)
Loaded 'Botnet-Friday-WorkingHours-Morning.pcap_ISCX' with shape (191033, 84)
Loaded 'Portscan-Friday-WorkingHours-Afternoon.pcap_ISCX' with shape (286467, 84)
Loaded 'DoS-Wednesday-WorkingHours.pcap_ISCX' with shape (692703, 84)
Loaded 'Infiltration-Thursday-WorkingHours-Afternoon.pcap_ISCX' with shape (288602, 84)

--- Initial Data Types ---

DDoS-Friday-WorkingHours-Afternoon.pcap_ISCX DataFrame dtypes:

Flow ID                          object
 Source IP                       object
 Source Port                      int64
 Destination IP                  object
 Destination Port                 int64
 Proto

DataFrames shrunk. Memory usage:
DDoS-Friday-WorkingHours-Afternoon.pcap_ISCX: 47.27 MB
WebAttacks-Thursday-WorkingHours-Morning.pcap_ISCX: 34.77 MB
Benign-Monday-WorkingHours.pcap_ISCX: 103.71 MB
Bruteforce-Tuesday-WorkingHours.pcap_ISCX: 88.83 MB
Botnet-Friday-WorkingHours-Morning.pcap_ISCX: 39.28 MB
Portscan-Friday-WorkingHours-Afternoon.pcap_ISCX: 25.53 MB
DoS-Wednesday-WorkingHours.pcap_ISCX: 130.55 MB
Infiltration-Thursday-WorkingHours-Afternoon.pcap_ISCX: 45.34 MB

Dropping constant columns...
  For DataFrame '', dropped 10 constant columns: ['Fwd Avg Packets/Bulk', 'Bwd Avg Bytes/Bulk', 'Bwd URG Flags', 'Fwd URG Flags', 'Bwd Avg Bulk Rate', 'Bwd PSH Flags', 'CWE Flag Count', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Packets/Bulk']
  For DataFrame '', dropped 10 constant columns: ['Fwd Avg Packets/Bulk', 'Bwd Avg Bytes/Bulk', 'Bwd URG Flags', 'Fwd URG Flags', 'Bwd Avg Bulk Rate', 'Bwd PSH Flags', 'CWE Flag Count', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Pac